In [1]:
import os
import pandas as pd
import numpy as np
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [2]:
def calcTaxPA(grossSalaryPA):
    '''calculates tax paid per year based on 
    UK tax bands'''
    taxFreeAllowance = 12500
    if grossSalaryPA <= taxFreeAllowance:
        taxPA = 0
    elif grossSalaryPA <= 50000:
        taxPA = 0.2*(grossSalaryPA - 12500)
    elif grossSalaryPA > 50000 and grossSalaryPA <= 150000:
        taxPA = (0.4*(grossSalaryPA - 50000)) + (0.2*(50000 - 12500))
    elif grossSalaryPA > 150000:
        taxPA = (0.45*(grossSalaryPA - 150000)) + (0.4*(150000 - 50000)) + (0.2*(50000 - 12500))
    return taxPA


def calcNICPA(grossSalaryPA):
    '''calculates national insurance contribution per year
    based on UK bands'''
    grossSalaryPW = grossSalaryPA/52
    if grossSalaryPW >= 962:
        NICPW = 0.02*(grossSalaryPW-962) + 0.12*(962-166)
    elif grossSalaryPW > 166:
        NICPW = 0.12*(grossSalaryPW - 166)
    else:
        NICPW = 0
    return NICPW*52


def calcStudentFinPA(grossSalaryPA):
    '''calculates student finance per year'''
    grossSalaryPM = grossSalaryPA/12
    thresholdPM = 2143
    if grossSalaryPM > thresholdPM:
        StudentFinPM = 0.09*(grossSalaryPM-thresholdPM)
    else:
        StudentFinPM = 0
    return StudentFinPM*12


def calcNetSalary(grossSalaryPA):
    '''calculates net salary after tax and all other deductions'''
    netSalaryPA = grossSalaryPA - calcTaxPA(grossSalaryPA) - calcNICPA(grossSalaryPA) - calcStudentFinPA(grossSalaryPA)
    return netSalaryPA


def salaryIncrease(grossSalaryPA, mean, sd):
    '''increase salary my random amount drawn from gaussian'''
    increaseStep = np.random.normal(mean, sd)
    return grossSalaryPA+increaseStep

def findInterestRate(paymentDate):
    year = paymentDate.year
    sdate = historicInterest[str(year)]["start"]
    edate = historicInterest[str(year)]["end"]
    delta = edate - sdate
    
    allDates = [sdate + timedelta(days=i) for i in range(delta.days+1)]

    if paymentDate in allDates:
        rate = historicInterest[str(year)]["rate"]
    else:
        rate = historicInterest[str(year-1)]["rate"]
    return rate

def graduation_amount(myPayments):
    for i, row in myPayments.iterrows():
    #     print(row)
        rate = findInterestRate(myPayments.at[i, "Payment Date"].date())
    
def populateInterestRate(myPayments):
    myPayments["interestRate"] = 0
    myPayments["totalPayment"] = 0
    myPayments["cumulativeTotal"] = 0
    for i, row in myPayments.iterrows():
    #     print(row)
        rate = findInterestRate(myPayments.at[i, "Payment Date"].date())
#         print(rate)
        myPayments.loc[i,"interestRate"] = rate
#         print(rate+1)
        myPayments.loc[i, "totalPayment"] = myPayments.loc[i, "Gross"]*(rate+1)
        myPayments.loc[i, "cumulativeTotal"] = myPayments.loc[i, "Gross"]*(rate+1)
    return myPayments

In [3]:
## build this dictionary from a table which is read in as a csv - potentially even scrape from the student finance website

historicInterest = {"2012": {"start": date(2012, 9, 1), "end": date(2013, 8, 31), "rate": 0.066}, 
                    "2013": {"start": date(2013, 9, 1), "end": date(2014, 8, 31), "rate": 0.063}, 
                    "2014": {"start": date(2014, 9, 1), "end": date(2015, 8, 31), "rate": 0.055}, 
                    "2015": {"start": date(2015, 9, 1), "end": date(2016, 8, 31), "rate": 0.039}, 
                    "2016": {"start": date(2016, 9, 1), "end": date(2017, 8, 31), "rate": 0.046}, 
                    "2017": {"start": date(2017, 9, 1), "end": date(2018, 8, 31), "rate": 0.0618}, 
                    "2018": {"start": date(2018, 9, 1), "end": date(2019, 8, 31), "rate": 0.063}
                   }


# incomeTax = {12500: 0.0, 50000: 0.2, 150000: 0.4, 0.45}
current_wd = os.getcwd()
file_name = "trimmed_data.csv"

def readPayments(file_name, date_header="Payment Date"):
    dataLoc = os.path.join(current_wd, "data", file_name)
    paymentsData = pd.read_csv(dataLoc)
    paymentsData[date_header] = pd.to_datetime(paymentsData[date_header])
    
    return paymentsData

In [39]:
## Read in data

myPayments = pd.read_csv("trimmed_data.csv", sep=",")
myPayments["Payment Date"] = pd.to_datetime(myPayments["Payment Date"])
myPayments

,Payment Date,Instalment,Gross
0,2014-09-25,1,1191.30
1,2014-10-15,1,2250.00
2,2015-01-05,2,1191.30
3,2015-02-04,2,2250.00
4,2015-04-20,3,1227.40
5,2015-05-06,3,4500.00
6,2015-09-24,4,1231.23
7,2015-10-21,4,2250.00
8,2016-01-04,5,1231.23
9,2016-02-03,5,2250.00


In [40]:
myPayments = populateInterestRate(myPayments)

In [41]:
myPayments

,Payment Date,Instalment,Gross,interestRate,totalPayment,cumulativeTotal
0,2014-09-25,1,1191.30,0.055,1256.82150,1256.82150
1,2014-10-15,1,2250.00,0.055,2373.75000,2373.75000
2,2015-01-05,2,1191.30,0.055,1256.82150,1256.82150
3,2015-02-04,2,2250.00,0.055,2373.75000,2373.75000
4,2015-04-20,3,1227.40,0.055,1294.90700,1294.90700
5,2015-05-06,3,4500.00,0.055,4747.50000,4747.50000
6,2015-09-24,4,1231.23,0.039,1279.24797,1279.24797
7,2015-10-21,4,2250.00,0.039,2337.75000,2337.75000
8,2016-01-04,5,1231.23,0.039,1279.24797,1279.24797
9,2016-02-03,5,2250.00,0.039,2337.75000,2337.75000


In [6]:
def simulateLifetimeEarnings(grossSalaryPA, N = 40):
    lifetime_tax = 0
    lifetime_SF = 0
    lifetime_NIC = 0
    lifetime_earning = 0
    
    for i in range(N):
#         print(i)
        if i%5 == 0 and i > 0 and i < 20:
            grossSalaryPA = salaryIncrease(grossSalaryPA, mean=5000, sd=3000)
        tax = calcTaxPA(grossSalaryPA)
        lifetime_tax += tax
        NIC = calcNICPA(grossSalaryPA)
        lifetime_NIC += NIC
        SF = calcStudentFinPA(grossSalaryPA)
        lifetime_SF += SF
        lifetime_earning += grossSalaryPA
        print("salary: ", grossSalaryPA, ". Student finance cumulative: ", lifetime_SF)
        if i == 30:
            print("***************30 YEARS***************")
        
    print("Across {} years of earning \nat a final gross salary of {} per year \nYou paid a total of: \n{} in tax \n{} in NIC \n{} in Student Finance".format(N, grossSalaryPA, lifetime_tax, lifetime_NIC, lifetime_SF))
    print("Across {} years of earning \nat you earned a cumulative gross salary of {}".format(N, lifetime_earning))

In [53]:
myPayments[myPayments.eq(date(2014, 10, 15)).any(1)]

,Payment Date,Instalment,Gross,interestRate,totalPayment,cumulativeTotal
1,2014-10-15,1,2250.0,0.055,2373.75,2373.75


In [ ]:
def graduandAmount(myPayments=None):
    cumulativeTotal = 0
    if myPayments is not None:
        startDate = min(myPayments["Payment Date"])
        endDate = max(["Payment Date"])
        while startDate < endDate:
            myPayments.
            cumulativeTotal *= (1+(interestRate/365))
            print("Date: ", startDate.strftime("%Y-%m-%d"), " || ", "Cumulative: ", cumulativeTotal, " || ", "Interest Rate: ", " || ", interestRate)
            startDate += delta
        
        interestRate = myPayments.at[i, "interestRate"]
            else:
                return cumulativeTotal
            delta = timedelta(days=1)
            cumulativeTotal += myPayments.at[i, "Gross"]
            
                
#                 print(cumulativeTotal)
    else:
        print("Please enter your net total of Student Debt at graduation")
        cumulativeTotal = input()
    return cumulativeTotal

cumulative = graduandAmount(myPayments)

In [48]:
def graduandAmount(myPayments=None):
    cumulativeTotal = 0
    if myPayments is not None:
        for i, row in myPayments.iterrows():
#             print(i)
            if i < myPayments.shape[0]-1:
                startDate = myPayments.at[i, "Payment Date"].date()
                endDate = myPayments.at[i+1, "Payment Date"].date()
                interestRate = myPayments.at[i, "interestRate"]
            else:
                return cumulativeTotal
            delta = timedelta(days=1)
            cumulativeTotal += myPayments.at[i, "Gross"]
            while startDate < endDate:
                cumulativeTotal *= (1+(interestRate/365))
                print("Date: ", startDate.strftime("%Y-%m-%d"), " || ", "Cumulative: ", cumulativeTotal, " || ", "Interest Rate: ", " || ", interestRate)
                startDate += delta
                
#                 print(cumulativeTotal)
    else:
        print("Please enter your net total of Student Debt at graduation")
        cumulativeTotal = input()
    return cumulativeTotal

cumulative = graduandAmount(myPayments)

Date:  2014-09-25  ||  Cumulative:  1191.4795109589043  ||  Interest Rate:   ||  0.055
Date:  2014-09-26  ||  Cumulative:  1191.659048967405  ||  Interest Rate:   ||  0.055
Date:  2014-09-27  ||  Cumulative:  1191.8386140295781  ||  Interest Rate:   ||  0.055
Date:  2014-09-28  ||  Cumulative:  1192.0182061495004  ||  Interest Rate:   ||  0.055
Date:  2014-09-29  ||  Cumulative:  1192.197825331249  ||  Interest Rate:   ||  0.055
Date:  2014-09-30  ||  Cumulative:  1192.3774715789018  ||  Interest Rate:   ||  0.055
Date:  2014-10-01  ||  Cumulative:  1192.557144896537  ||  Interest Rate:   ||  0.055
Date:  2014-10-02  ||  Cumulative:  1192.7368452882338  ||  Interest Rate:   ||  0.055
Date:  2014-10-03  ||  Cumulative:  1192.9165727580719  ||  Interest Rate:   ||  0.055
Date:  2014-10-04  ||  Cumulative:  1193.0963273101315  ||  Interest Rate:   ||  0.055
Date:  2014-10-05  ||  Cumulative:  1193.2761089484934  ||  Interest Rate:   ||  0.055
Date:  2014-10-06  ||  Cumulative:  1193.45591

Date:  2015-10-07  ||  Cumulative:  14303.48563281552  ||  Interest Rate:   ||  0.039
Date:  2015-10-08  ||  Cumulative:  14305.013950458479  ||  Interest Rate:   ||  0.039
Date:  2015-10-09  ||  Cumulative:  14306.542431401129  ||  Interest Rate:   ||  0.039
Date:  2015-10-10  ||  Cumulative:  14308.071075660922  ||  Interest Rate:   ||  0.039
Date:  2015-10-11  ||  Cumulative:  14309.599883255307  ||  Interest Rate:   ||  0.039
Date:  2015-10-12  ||  Cumulative:  14311.128854201736  ||  Interest Rate:   ||  0.039
Date:  2015-10-13  ||  Cumulative:  14312.657988517663  ||  Interest Rate:   ||  0.039
Date:  2015-10-14  ||  Cumulative:  14314.187286220546  ||  Interest Rate:   ||  0.039
Date:  2015-10-15  ||  Cumulative:  14315.716747327839  ||  Interest Rate:   ||  0.039
Date:  2015-10-16  ||  Cumulative:  14317.246371857005  ||  Interest Rate:   ||  0.039
Date:  2015-10-17  ||  Cumulative:  14318.776159825504  ||  Interest Rate:   ||  0.039
Date:  2015-10-18  ||  Cumulative:  14320.30

Date:  2016-10-06  ||  Cumulative:  27832.387490148867  ||  Interest Rate:   ||  0.046
Date:  2016-10-07  ||  Cumulative:  27835.89513350379  ||  Interest Rate:   ||  0.046
Date:  2016-10-08  ||  Cumulative:  27839.403218917876  ||  Interest Rate:   ||  0.046
Date:  2016-10-09  ||  Cumulative:  27842.911746446836  ||  Interest Rate:   ||  0.046
Date:  2016-10-10  ||  Cumulative:  27846.42071614639  ||  Interest Rate:   ||  0.046
Date:  2016-10-11  ||  Cumulative:  27849.93012807226  ||  Interest Rate:   ||  0.046
Date:  2016-10-12  ||  Cumulative:  27853.439982280182  ||  Interest Rate:   ||  0.046
Date:  2016-10-13  ||  Cumulative:  27856.950278825894  ||  Interest Rate:   ||  0.046
Date:  2016-10-14  ||  Cumulative:  27860.461017765145  ||  Interest Rate:   ||  0.046
Date:  2016-10-15  ||  Cumulative:  27863.972199153686  ||  Interest Rate:   ||  0.046
Date:  2016-10-16  ||  Cumulative:  27867.483823047278  ||  Interest Rate:   ||  0.046
Date:  2016-10-17  ||  Cumulative:  27870.9958

In [49]:
cumulative

38833.90648170636

In [15]:
startDate = min(myPayments["Payment Date"]).date
delta
newDate = startDate + timedelta(days=1)

TypeError: unsupported operand type(s) for +: 'builtin_function_or_method' and 'datetime.timedelta'

# To-do
* Function to calculate student finance owed - need this as separate so can calculate cumulative total over simulated years 

In [8]:
calcTaxPA(35000-(0.05*35000))/12
calcNICPA(35000-(0.05*35000))/12
calcStudentFinPA(35000-(0.05*35000))/12

56.50500000000002

In [9]:
# interest = 
principal = 1191.3

final = compoundInterest(P = principal, r = 0.055/365, n = 1, t=5)
final-principal

NameError: name 'compoundInterest' is not defined

In [ ]:
(1191.3*(0.055/365)*20)+(2250*(0.055/365)*15)

In [10]:
principal = 1191.3
for i in range(5):
    principal = principal*(1+(0.055/365))

print(principal-1191.3)

second = principal+2250
(0.055/365)*second*15

0.8978253312491233


7.780310153145975

In [11]:
ans_1 = (1+(5*0.055/365))*1191.3
ans = ans_1*(1+(15*0.055/365))
ans = ans+(1+(15*0.055/365))*2250
ans

3449.9778643361797

In [12]:
ans = (1+(15*0.055/365))*2250
ans - 2250

5.085616438356283

In [13]:
dailyRate = 0.055/364
amountToday = 1191.3*(1+dailyRate)
interestAccumulated = 0
for i in range(16):
    interestToday = dailyRate*amountToday
    amountToday = amountToday+interestToday
    interestAccumulated += interestToday
amountToday = amountToday + 2250
for i in range(15):
    interestToday = dailyRate*amountToday
    amountToday = amountToday+interestToday
    interestAccumulated += interestToday
print(interestAccumulated)

10.698623812453368


In [18]:
def compoundInterest(P, r, n, t):
    return P*(1+(r/n))**(t)

In [23]:
principal = 1191.3
final = compoundInterest(P=principal, r=0.055, n=365,t=1)
print(final-principal)

0.7182061495007019


In [49]:
final = final*(1+(0.054/365))

In [50]:
final-1191.3

0.8814964041873736

In [36]:
principal = 1191.3
final = principal*(1+(0.055/365))
final*=(1+(0.055/365))
final*=(1+(0.055/365))
final*=(1+(0.055/365))
final*=(1+(0.055/365))
print(final-principal)

0.8978253312491233
